# GenU - Cognito ユーザプールにユーザを一括登録したい
参考
- [Confirmed状態のCognitoユーザーをBoto 3で即作成する](https://qiita.com/ttkiida/items/fdd93e166f2d36e581e4)
- [boto3 - CognitoIdentityProvider - admin_create_user](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cognito-idp/client/admin_create_user.html)

In [1]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [53]:
import boto3
import pandas as pd
from tqdm.notebook import tqdm


def get_all_users(user_pool_id):
    """ Cognitoユーザプールからユーザリストを取得
    Args:
        user_pool_id (string): CognitoユーザプールID
    Returns:
        users (list): ユーザプール内の全てのユーザリスト
    """
    cognito_client = boto3.client('cognito-idp')
    users = []
    pagination_token = None

    while True:
        if pagination_token:
            response = cognito_client.list_users(
                UserPoolId=user_pool_id,
                PaginationToken=pagination_token
            )
        else:
            response = cognito_client.list_users(
                UserPoolId=user_pool_id
            )

        users.extend(response['Users'])

        if 'PaginationToken' in response:
            pagination_token = response['PaginationToken']
        else:
            break

    return users


def users_to_dataframe(users):
    """ Cognitoユーザプールから取得したユーザリストをデータフレームに変換
    Args:
        users (list): ユーザプール内の全てのユーザリスト
    Returns:
        (pd.DataFrame): Cognitoのユーザリスト
    """
    user_data = []
    for user in users:
        user_dict = {
            'Username': user['Username'],
            'UserStatus': user['UserStatus'],
            'UserCreateDate': user['UserCreateDate'].strftime('%Y-%m-%d %H:%M:%S'),
            'UserLastModifiedDate': user['UserLastModifiedDate'].strftime('%Y-%m-%d %H:%M:%S'),
        }

        # Add user attributes
        for attr in user['Attributes']:
            user_dict[attr['Name']] = attr['Value']

        user_data.append(user_dict)

    return pd.DataFrame(user_data)


def check_string_in_column(df, column_name, string_to_check):
    """
    Args:
        df(Dataframe): チェック対象のデータフレーム
        column_name(str): チェック対象の列名
        string_to_check(str): 検索する文字列
    Return:
        (list): データフレームの各行について、True|False のリスト
    """
    return df[column_name].str.contains(string_to_check, na=False)


def create_cognito_users(USER_POOL_ID, df):
    """ Cognito ユーザプールに新規ユーザをまとめて作成する
    Args:
        USER_POOL_ID (str): CognitoユーザプールID
        df (pd.DataFrame): 登録したいユーザと初期パスワードをまとめたデータフレーム
    """
    cognito_client = boto3.client('cognito-idp')

    # Cognito ユーザプールから最新のユーザリストを取得
    user_list = get_all_users(USER_POOL_ID)
    # print(f"Cognito ユーザプールに登録されている人数: {len(user_list)}")

    # ユーザリストをデータフレームに変換
    df_users = users_to_dataframe(user_list)

    # 一時的な仮パスワード
    PASS_TEMP = "Karipass#9020"

    # Admin権限でユーザを新規作成する
    for index, row in tqdm(df.iterrows(), total=len(df)):
        EMAIL = row["メールアドレス"]
        PASS = row["初期パスワード"]

        # Cognitoユーザプール内に重複が無いかをチェック
        if check_string_in_column(df_users, 'email', EMAIL).sum():
            print(f'Creatig user >>> {EMAIL} >>> pass')
            continue
        print(f'Creatig user >>> {EMAIL}')

        # 共通の仮パスワードを使って、とりあえずユーザ作成
        # ここで一時パスワードを設定すると時限付となってしまう
        # メール認証は `済` にしてしまう
        cognito_client.admin_create_user(
            UserPoolId=USER_POOL_ID,
            Username=EMAIL,
            TemporaryPassword=PASS_TEMP,
            UserAttributes=[
                {
                    'Name': 'email_verified',
                    'Value': 'true'
                },
                {
                    'Name': 'email',
                    'Value': EMAIL
                }
            ],
            MessageAction='SUPPRESS'
        )

        # 初期パスワードを確認済みとして設定
        cognito_client.admin_set_user_password(
            UserPoolId=USER_POOL_ID,
            Username=EMAIL,
            Password=PASS,
            Permanent=True
        )

    print("New user created in the Cognito user pool.")

print("準備OK")

準備OK


## 登録したいユーザ情報を取得する

In [50]:
df = pd.read_excel('GenU_一括登録リスト（テスト）.xlsx')
df = df.dropna(subset='メールアドレス')
df = df.filter(['メールアドレス', '初期パスワード'])
df

,メールアドレス,初期パスワード
0,temp-east@jreast.co.jp,Test_90201534


In [51]:
# 必要により実行
print("データのチェック")
for index, row in df.iterrows():
    print(f'{index}>>> {row["メールアドレス"]} | {row["初期パスワード"]}')

データのチェック
0>>> temp-east@jreast.co.jp | Test_90201534


# データフレームから Cognito ユーザを作成する

In [54]:
# 一括登録リストにあるユーザをまとめて作成する
USER_POOL_ID = 'ap-northeast-1_Ln02xdsyw'    # Cognito ユーザプールID
create_cognito_users(USER_POOL_ID, df)

  0%|          | 0/1 [00:00<?, ?it/s]

Creatig user >>> temp-east@jreast.co.jp
New user created in the Cognito user pool.


# 仮パスワードの期限切れへの対応

## とりあえず試してみる（作業用）

In [3]:
import boto3

cognito_client = boto3.client('cognito-idp')

In [4]:
USER_POOL_ID = 'ap-northeast-1_Ln02xdsyw'    # Cognito ユーザプールID

In [6]:
users = get_all_users(USER_POOL_ID)

In [8]:
df_users = users_to_dataframe(users)

In [10]:
df_users['UserStatus'].unique()

array(['FORCE_CHANGE_PASSWORD', 'CONFIRMED', 'UNCONFIRMED'], dtype=object)

In [12]:
df_users[df_users['UserStatus'] == 'FORCE_CHANGE_PASSWORD']

,Username,UserStatus,UserCreateDate,UserLastModifiedDate,sub,email_verified,email
0,0057dc56-79b8-42fe-bfca-bdac823fc950,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:40,2024-09-09 07:13:40,0057dc56-79b8-42fe-bfca-bdac823fc950,true,kurayoshi@jreast.co.jp
5,05e5a17c-a7ef-48bb-9fde-dbac972f9494,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:48,2024-09-09 07:13:48,05e5a17c-a7ef-48bb-9fde-dbac972f9494,true,ta-hirose@jreast.co.jp
11,0a62151f-d8ff-4932-ad43-5ad16b9793be,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:43,2024-09-09 07:13:43,0a62151f-d8ff-4932-ad43-5ad16b9793be,true,kousaku-hatakeyama@jreast.co.jp
12,0a808bdf-c9aa-4b1b-b526-1c309a164475,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:49,2024-09-09 07:13:49,0a808bdf-c9aa-4b1b-b526-1c309a164475,true,tomohiro-watanabe@jreast.co.jp
13,0abe8ae6-04f9-46be-9f4a-167fb472dcc9,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:50,2024-09-09 07:13:50,0abe8ae6-04f9-46be-9f4a-167fb472dcc9,true,yuuki-satou@jreast.co.jp
...,...,...,...,...,...,...,...
314,f15ce09b-e1a8-4403-b623-053146eb81d4,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:49,2024-09-09 07:13:49,f15ce09b-e1a8-4403-b623-053146eb81d4,true,ushio@jreast.co.jp
316,f1b0c6de-0de2-4441-bae4-78ae8ceb6dfa,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:46,2024-09-09 07:13:46,f1b0c6de-0de2-4441-bae4-78ae8ceb6dfa,true,inkyo-eiji@jreast.co.jp
317,f312c2fb-a7bb-4e56-95c0-04d3445e25de,FORCE_CHANGE_PASSWORD,2024-09-09 07:13:48,2024-09-09 07:13:48,f312c2fb-a7bb-4e56-95c0-04d3445e25de,true,y-paku@jreast.co.jp
319,f60936c2-1753-42a6-a541-7dfe793e7183,FORCE_CHANGE_PASSWORD,2024-09-09 06:55:28,2024-09-09 06:55:28,f60936c2-1753-42a6-a541-7dfe793e7183,true,kazuma-moriya@jreast.co.jp


In [14]:
df_users[df_users['email'] == 'd-ueda@jreast.co.jp']

,Username,UserStatus,UserCreateDate,UserLastModifiedDate,sub,email_verified,email
258,c3e17781-a3c1-4319-b04e-baab8fff6916,FORCE_CHANGE_PASSWORD,2024-09-09 06:55:26,2024-09-09 06:55:26,c3e17781-a3c1-4319-b04e-baab8fff6916,true,d-ueda@jreast.co.jp


In [27]:
EMAIL = 'd-ueda@jreast.co.jp'

PASS = df.query(f'メールアドレス == "{EMAIL}"')['初期パスワード'].iloc[0]
PASS

'D-ueda_32131'

In [29]:
response = cognito_client.admin_set_user_password(
    UserPoolId=USER_POOL_ID,
    Username='d-ueda@jreast.co.jp',
    Password=PASS,
    Permanent=True
)

In [30]:
response

{'ResponseMetadata': {'RequestId': '1278da7e-9a90-4b70-94bc-23e3a343a3b5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 17 Sep 2024 04:16:19 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1278da7e-9a90-4b70-94bc-23e3a343a3b5'},
  'RetryAttempts': 0}}

## まとめて、確認済みのパスワードを設定する

In [40]:
import boto3
from tqdm.notebook import tqdm


# cognito_client = boto3.client('cognito-idp')
USER_POOL_ID = 'ap-northeast-1_Ln02xdsyw'    # Cognito ユーザプールID

# 登録対象のユーザリスト
users = get_all_users(USER_POOL_ID)
df_users = users_to_dataframe(users)
target_users = df_users[df_users['UserStatus'] == 'FORCE_CHANGE_PASSWORD']['email'].to_list()

# 確認済みのパスワードを設定する
for USER in tqdm(target_users):
    print(f"{USER} >>> set parmanent password")
    PASS = df.query(f'メールアドレス == "{USER}"')['初期パスワード'].iloc[0]
    response = cognito_client.admin_set_user_password(
        UserPoolId=USER_POOL_ID,
        Username=USER,
        Password=PASS,
        Permanent=True
    )

  0%|          | 0/134 [00:00<?, ?it/s]

kurayoshi@jreast.co.jp >>> set parmanent password
ta-hirose@jreast.co.jp >>> set parmanent password
kousaku-hatakeyama@jreast.co.jp >>> set parmanent password
tomohiro-watanabe@jreast.co.jp >>> set parmanent password
yuuki-satou@jreast.co.jp >>> set parmanent password
k-miyaki@jreast.co.jp >>> set parmanent password
t-maru@jreast.co.jp >>> set parmanent password
igarashi-a@jreast.co.jp >>> set parmanent password
ujiie-yu@jreast.co.jp >>> set parmanent password
chikara-saitou@jreast.co.jp >>> set parmanent password
toru-matsuda@jreast.co.jp >>> set parmanent password
n-ichinohe@jreast.co.jp >>> set parmanent password
onouchi@jreast.co.jp >>> set parmanent password
shige-yoshida@jreast.co.jp >>> set parmanent password
yasuyuki-takahashi@jreast.co.jp >>> set parmanent password
kawabata@jreast.co.jp >>> set parmanent password
mozawa-k@jreast.co.jp >>> set parmanent password
youko-satou@jreast.co.jp >>> set parmanent password
nakatao@jreast.co.jp >>> set parmanent password
norihisa-yamaguch